In [22]:
import pandas as pd
import json
import os
import gpxo
import gpxpy
import gpxpy.gpx
import fitdecode
import warnings
import datetime
import numpy as np

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

### Feature engineering para arquivos Json

In [23]:
def json_processed_data(folder_path):

    '''Função para ler todas as atividades do atleta com arquivos no formato json e a partir disso retornar um conjunto com as features criadas'''

    # Contador de arquivos dentro da pasta
    files_count = 0

    for file in os.listdir(folder_path):
        files_count += 1

    # Caminho do arquivo de atividade com {} para inserir o número
    # Nome do arquivo no formato: atividades(x).json onde x >= n.
    filename = "atividades ({}).json"

    full_path = os.path.join(folder_path, filename)  
    n = 1 # Se as atividades começarem por ex com 0, então mudar para n = 0.

    # Leitura de cada uma das atividades no formato json com json.load. 
    activities_data = [json.load(open(full_path.format(activity),"rb")) for activity in range(n, files_count + n)]

    df = pd.DataFrame(activities_data)

    # Lista das metricas que serão identificadas
    metrics = ['distance', 'pace', 'speed']

    # Dicionário com as features que serão criadas com o valor de cada atividade
    df_athlete_dict = {
        'total_distance (km)': [], 
        'pace (min/km)': [],
        'velocity (km/h)': []    
    }

    # Iteração no número de atividades
    for activity in range(len(df)):
        # Iteração para encontrar a linha com a métrica desejada e adicionar seu valor no dicionário com a chave certa
        for values in df['summaries'][activity]: # por atividade
            if (values['metric'] == metrics[0]): 
                df_athlete_dict['total_distance (km)'].append(round(values['value'],2))
            elif (values['metric'] == metrics[1]):
                df_athlete_dict['pace (min/km)'].append(round(values['value'],2))
            elif (values['metric'] == metrics[2]):
                df_athlete_dict['velocity (km/h)'].append(round(values['value'],2))

    # Criação do DataFrame de atividades
    df_athlete_activities = pd.DataFrame(df_athlete_dict)

    # Tempo inicial da atividade
    start = pd.to_datetime(df['start_epoch_ms'],unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('s')

    # Tempo final da atividade
    end = pd.to_datetime(df['end_epoch_ms'],unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('s')

    # Criação da coluna com tempo total em minutos a partir da subtração dos tempos final e inicial
    df_athlete_activities['total_time (min)'] = (end - start).apply(lambda x: x.total_seconds() // 60)

    # Criando a coluna para a data da atividade
    df_athlete_activities.insert(0, 'activity_date', pd.to_datetime(df['end_epoch_ms'],unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('d'))

    return df_athlete_activities

### Feature engineering para arquivos Gpx

In [24]:
def gpx_processed_data(folder_path):

    '''Função para ler todas as atividades do atleta com arquivos no formato gpx e a partir disso retornar um conjunto com as features criadas'''

    # Dicionário com as features que serão criadas com o valor de cada atividade
    df_athlete_dict = {
        'activity_date': [],
        'total_distance (km)': [], 
        'total_time (min)': [],
        'pace (min/km)': [],
        'velocity (km/h)': []    
    }

    # Itera por cada arquivo na pasta
    for filename in os.listdir(folder_path):
        # Junta o nome do arquivo com o caminho da pasta
        full_path = os.path.join(folder_path, filename)

        # Lê o arquivo no formato gpx com a biblioteca gpxo e transforma em df
        gpx = gpxo.Track(full_path).data.reset_index()    

        '''Cocantenação de elementos novos para cada feature no df, a cada iteração'''

        # Data da atividade (a partir da primeira linha do df original com os pontos marcados)
        df_athlete_dict['activity_date'].append(gpx['time'][0].round('d'))

        # Distância total em km
        total_dist = gpx['distance (km)'].max()
        df_athlete_dict['total_distance (km)'].append(round(total_dist,2))

        # Tempo total em minutos
        total_min = (gpx['time'].max() - gpx['time'].min()).total_seconds() // 60
        df_athlete_dict['total_time (min)'].append(total_min)

        # Ritmo geral da atividade
        df_athlete_dict['pace (min/km)'].append(round(total_min / total_dist, 2))

        # Velocidade em km/h
        df_athlete_dict['velocity (km/h)'].append(round(total_dist / (total_min / 60),2))
    
    # Passando o dicionário com as features criadas para um Dataframe
    df_athlete_activities = pd.DataFrame(df_athlete_dict)
    
    return df_athlete_activities

### Feature engineering para arquivos Fit

In [25]:
def fit_processed_data(folder_path):
    '''Função para ler todas as atividades do atleta com arquivos no formato fit e a partir disso retornar um conjunto com as features criadas'''

    # Dicionário com as features que serão criadas com o valor de cada atividade
    df_athlete_dict = {
        'activity_date': [],
        'total_distance (km)': [], 
        'total_time (min)': [],
        'pace (min/km)': [],
        'velocity (km/h)': []    
    }

    # Itera por cada arquivo na pasta
    for filename in os.listdir(folder_path):
        # Junta o nome do arquivo com o caminho da pasta
        full_path = os.path.join(folder_path, filename)

        # Acessando o arquivo fit com função de leitura para obter os dados
        with fitdecode.FitReader(full_path) as fit_file:
            # Iterando for cada frame no arquivo fit
            for frame in fit_file:
                # Condição para o frame que contém os dados gps
                if frame.frame_type == fitdecode.FIT_FRAME_DATA:
                    # Condição para acessar o frame envolvendo métricas resumidas da sessão
                    if frame.name == 'session':
                        # Iteração pelos campos que possuem as métricas
                        for field in frame.fields:
                            # Condição para acessar o campo de start_time
                            if field.name == 'start_time':
                                df_athlete_dict['activity_date'].append(field.value)

                            # Condição para acessar o campo de total_distance
                            elif field.name == 'total_distance':
                                total_dist_km = field.value / 1000
                                df_athlete_dict['total_distance (km)'].append(round((total_dist_km),2))

                            # Condição para acessar o campo de total_timer_time
                            elif field.name == 'total_timer_time':
                                total_time_min = field.value // 60
                                df_athlete_dict['total_time (min)'].append(total_time_min)

        # Condição para caso a distância seja 0, por causa da divisão
        if total_dist_km == 0:
            df_athlete_dict['pace (min/km)'].append(0)
        else:    
            df_athlete_dict['pace (min/km)'].append(round((total_time_min / total_dist_km),2))

        df_athlete_dict['velocity (km/h)'].append(round(total_dist_km / (total_time_min / 60),2))
    
    # Passando o dicionário com as features criadas para um Dataframe
    df_athlete_activities = pd.DataFrame(df_athlete_dict)

    # Convertendo a coluna de data para o fuso horário local e arrendado em dias
    df_athlete_activities['activity_date'] = df_athlete_activities['activity_date'].dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('d')

    return df_athlete_activities

### Feature engineering of time features

In [26]:
def activities_time_frequency(df_athlete_activities):

    # Dropando linhas com datas duplicadas
    df_athlete_activities.drop_duplicates(subset='activity_date', inplace= True)

    # Ordenando a data da menor pra maior e resetando index 
    df_athlete_activities.sort_values('activity_date', ascending=True, inplace=True)    
    df_athlete_activities.reset_index(drop=True, inplace=True)

    # Criando uma coluna para o tipo de atividade e designando todas como Atividade
    df_athlete_activities.insert(1, 'activity_type', 'Activity')

    # Fazendo a diferença entre a atividade seguinte e a anterior e preenchendo a primeira linha com 0
    df_athlete_activities['days_between_activities'] = df_athlete_activities['activity_date'].diff(periods=1).fillna(datetime.timedelta(days=1))

    # Alterando o tipo da coluna para int
    df_athlete_activities['days_between_activities'] = (df_athlete_activities['days_between_activities'].dt.days) - 1

    # Data da primeira atividade
    start = df_athlete_activities['activity_date'].iloc[0]

    # Data da última atividade
    end = df_athlete_activities['activity_date'].iloc[-1]

    # Dias entre a primeira e última atividade
    dates = pd.date_range(start, end, freq='d').round('d')

    # Colocando todas as datas em um df
    df_dates = pd.DataFrame({'activity_date': dates})

    # Right join pra poder manter os dados das atividades existentes mas adicionar linhas para os novos dias
    df_athlete_activities = df_dates.merge(df_athlete_activities, on='activity_date', how='left')

    # Preechendo os valores nulos como Dia sem atividade
    df_athlete_activities['activity_type'].fillna('No Activity', inplace=True)

    # Preenchendo os valores NaN com 1 para poder fazer uma subtração cumulativa
    df_athlete_activities['days_between_activities'].fillna(-1, inplace= True)

    # Fazendo cumsum com valores negativos e depois multiplicando por -1 pra transformá-los em positivos
    df_athlete_activities['days_between_activities'] = df_athlete_activities['days_between_activities'].cumsum() * -1

    # Transformando em int pra evitar -0 ao invés de 0
    df_athlete_activities['days_between_activities'] = df_athlete_activities['days_between_activities'].astype(int)

    return df_athlete_activities

### Drift Detection

In [27]:
def drift_detection(df_example, drift_method_obj, results, param, drift_method, x, param_combination_count, example_id):
    # Lista para obter todos os indexes que houver drift
    drifts = []
    df = df_example.copy()

    # Index da ultima atividade do atleta antes da desistência/inatividade física
    last_activity_index = df[df['activity_type'] == 1].iloc[-1].name    

    # Range de x dias antes da última atividade
    range_x_before_activity = [last_activity_index - value if last_activity_index >= value else 0 for value in range(0, x + 1)]
    # Range de x dias depois da última atividade
    range_x_after_activity = [last_activity_index + value for value in range(1, x + 1)]
    # Lista com o range de indexes possíveis para classificar um drift como TP
    drift_range = list(set(range_x_before_activity + range_x_after_activity))
    # Ordenação da lista de range de indexes do menor para o maior
    drift_range.sort()

    # Criando coluna no df com classificação 1 para todos os indexes que estiverem dentro do range de drift
    df.loc[drift_range,'drift_possibility_range'] = 1
    # Classificando como 0 os indexes fora do range
    df['drift_possibility_range'].fillna(0, inplace=True)

    # Iteração pelo df atualizado a cada linha para encontrar drifts
    for index, row in df.iterrows():
            drift_method_obj.update(row['days_between_activities'])
            if drift_method_obj.drift_detected:
                drifts.append(index)
    
    # Criando coluna no df com classificação 1 para onde houver drift 
    df.loc[drifts,'drift_classification'] = 1
    # Classificando como 0 onde não houve drift
    df['drift_classification'].fillna(0, inplace=True)

    # Condição para filtrar uma linha que haja um drift TP
    TP_condition = (df['drift_possibility_range'] == 1) & (df['drift_classification'] == 1)

    # Condição para caso haja um ou mais drift TP
    if (TP_condition).sum() != 0:
        # Selecionando o primeiro drift TP
        first_TP_drift = df.loc[TP_condition, 'activity_date'].iloc[0]
        # Selecionando a última linha (Onde ocorreu a desistência/Inatividade física)
        end = df['activity_date'].iloc[-1]
        # Obtendo a quantidade de dias em que foi previsto o primeiro drift TP antes da desistência/Inatividade física
        days_of_first_DD_before_PI = (end - first_TP_drift).days
    else:
        # Caso não haja drift TP, a variável possui valor NaN
        days_of_first_DD_before_PI = np.nan

    # Adicionando na lista results a avaliação do drift detection
    evaluation = evaluate(df, param, drift_method, days_of_first_DD_before_PI, param_combination_count, example_id)
    results.append(evaluation) 

### Drift Detection Evaluation

In [28]:
def evaluate(df, drift_params, drift_method, days_of_first_DD_before_PI, param_combination_count, example_id):
    # Definição de TP, FP, TN e FN como zero, inicialmente
    TP = FP = TN = FN = 0 
    
    # Iteração pelo df a cada linha para classificar como TP, TN ou FP
    for index, row in df.iterrows():
        if (row['drift_possibility_range'] == 0) & (row['drift_classification'] == 0):
            TN +=1

        elif (row['drift_possibility_range'] == 0) & (row['drift_classification'] == 1):
            FP +=1

        elif (row['drift_possibility_range'] == 1) & (row['drift_classification'] == 1):
            TP +=1
    # Caso o TP fosse 0, significa que houve FN
    if (TP == 0):
        FN = 1
    # Caso o TP fosse >= 1, não houve FN pois o(s) drift(s) foram detectados
    elif (TP >= 1):
        FN = 0
    
    # Valores que serão retornados da função
    return {
        'Parameter Combination id': param_combination_count,
        'Example id': example_id,
        'Example': df,
        'Drift Parameters': drift_params,
        'Drift Method': drift_method,
        'Days of First DD before PI': days_of_first_DD_before_PI,
        'TP': TP,
        'TN': TN,
        'FP': FP,
        'FN': FN,
    }      

In [29]:
def metrics(TP, FP, TN, FN):

    # Calcular a precisão
    if (TP + FP) != 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    
    # Calcular o recall
    if (TP + FN) != 0:
        recall = TP / (TP + FN)
    else:
        recall = 0
    
    # Calcular o F1-score
    if (precision + recall) != 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0
    
    return round(precision,2), round(recall,2), round(f1,2)